# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook

%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues


# APK Files to load
file_paths = (
#     "../../../Downloads/com.snapchat.android_10.87.5.69-2074_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.88.0.63-2076_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.88.5.72-2077_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.89.0.66-2078_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.89.7.78-2085_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_11.0.0.72-2086_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_11.0.1.72-2089_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.snapchat.android_11.0.6.82-2095_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.snapchat.android_11.1.1.66-2097_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)


# file_paths = (
# #     "../../../Downloads/com.instagram.android_143.0.0.25.121-216817305_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_151.0.0.23.120-232868015_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_151.0.0.23.120-232867993_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847234_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847224_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_153.0.0.34.96-236572355_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_153.0.0.34.96-236572319_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_156.0.0.26.109-240726452_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_156.0.0.26.109-240726384_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
#     "../../../Downloads/com.instagram.android_157.0.0.37.120-242168935_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.instagram.android_157.0.0.37.120-242168922_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"
# )

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.snapchat.android_11.0.6.82-2095_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk
Loading Session from Apk at ../../../Downloads/com.snapchat.android_11.1.1.66-2097_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk


# List of Classes, Methods and Variables

Defining Elements that GraphGuard should try to match (obviously requires full class names).
* `named_c_decs` in form of a tuple of Strings
* `named_m_decs` in form of a tuple of MethodDeclarations
* `named_f_decs` in form of a tuple of FieldDeclarations

In [5]:
# Classes of the Methods and Fields are automatically merged with c_decs

named_c_decs = {
    'SCREENSHOT_DETECTOR': 'DR5',
    'STORY_VIEWED_PLUGIN': 'kuh',
    'SNAP_MODEL': 'qdc',
    'APP_START_EXPERIMENT_MANAGER': 'cUe',
    'CBC_ENCRYPTION_ALGORITHM': 'zY6'
}

In [6]:
named_m_decs = {
    'SCREENSHOT_DETECTED': MethodDec('DR5', 'a', 'DR5', 'CR5'),
    'MARK_STORY_AS_VIEWED': MethodDec('kuh', 'W', 'kce', 'V1e'),
    'SNAP_MODEL_CONSTRUCTOR': MethodDec('qdc', '<init>', 'java.lang.String', 'boolean', 'java.lang.String', 'java.lang.String', 'java.lang.String', 'java.lang.Long', 'iG5', 'long', 'boolean', 'java.lang.Long', 'long'),
    'FORCE_APP_DECK': MethodDec('cUe', 'f', 'e95'),
    'DECRYPT_MEDIA_STREAM': MethodDec('zY6', 'y0', 'java.io.InputStream')
}

In [7]:
named_f_decs = {

}

In [8]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [9]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [10]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [11]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [12]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Method. Considering it a match 
	DR5#a(DR5, CR5) -> CV5#a(CV5, BV5)
* Found multiple candidates for Method kuh#W(kce, V1e)
+ Matching Class of single candidate method match 
	LDR5; -> LCV5;
Could resolve 1 new Classes, 1 new Methods, 0 new Fields.


### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [13]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LcUe; -> Lz3f;
+ Found single candidate for Class. Considering it a match! 
	LzY6; -> Lq47;
+ Found single candidate for Class. Considering it a match! 
	Lkuh; -> LUEh;
Could resolve 3 new Classes, 0 new Methods, 0 new Fields.


## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [14]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	Lqdc; -> Ljmc;
Could resolve 1 new Classes, 0 new Methods, 0 new Fields.


## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [15]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

+ Found single candidate for Method. Considering it a match 
	kuh#W(kce, V1e) -> UEh#Y(Cke, oae)
+ Found single candidate for Method. Considering it a match 
	qdc#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, iG5, long, boolean, java.lang.Long, long) -> jmc#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, cK5, long, boolean, java.lang.Long, long)
+ Found single candidate for Method. Considering it a match 
	zY6#y0(java.io.InputStream) -> q47#y0(java.io.InputStream)
Could resolve 0 new Classes, 3 new Methods, 0 new Fields.

Non-Exact Matching

+ Found single candidate for Method. Considering it a match 
	cUe#f(e95) -> z3f#f(Jc5)
Could resolve 0 new Classes, 1 new Methods, 0 new Fields.


## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [16]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [17]:
print(len(accumulator.matching_cs), "/", len(named_c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

5 / 5 Classes were matched
5 / 5 Methods were matched
0 / 0 Fields were matched

Matching Classes:
• DR5 -> CV5
• cUe -> z3f
• zY6 -> q47
• kuh -> UEh
• qdc -> jmc

Matching Methods:
• DR5#a(DR5, CR5) -> CV5#a(CV5, BV5)
• kuh#W(kce, V1e) -> UEh#Y(Cke, oae)
• qdc#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, iG5, long, boolean, java.lang.Long, long) -> jmc#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, cK5, long, boolean, java.lang.Long, long)
• zY6#y0(java.io.InputStream) -> q47#y0(java.io.InputStream)
• cUe#f(e95) -> z3f#f(Jc5)

Matching Fields:


In [18]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(named_c_decs.values(), r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('CV5', 'a', 'CV5', 'BV5'),
    MethodDec('UEh', 'Y', 'Cke', 'oae'),
    MethodDec('jmc', '<init>', 'java.lang.String', 'boolean', 'java.lang.String', 'java.lang.String', 'java.lang.String', 'java.lang.Long', 'cK5', 'long', 'boolean', 'java.lang.Long', 'long'),
    MethodDec('z3f', 'f', 'Jc5'),
    MethodDec('q47', 'y0', 'java.io.InputStream'),
)
c_decs = (
    'CV5'
    'UEh'
    'jmc'
    'z3f'
    'q47',
)
f_decs = (
)


In [19]:
c_file = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/ClassDeclarations.kt"
m_file = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/MemberDeclarations.kt"
f_file = "/dev/null"


"""
c_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
m_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
f_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"
"""

'\nc_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"\nm_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"\nf_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"\n'

In [20]:
%autoreload

c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

In [21]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.jaqxues.sniptools.packimpl.hookdec

import com.jaqxues.akrolyb.genhook.decs.ClassDec

object ClassDeclarations {
    val SCREENSHOT_DETECTOR = ClassDec("CV5")
    val STORY_VIEWED_PLUGIN = ClassDec("UEh")
    val SNAP_MODEL = ClassDec("jmc")
    val APP_START_EXPERIMENT_MANAGER = ClassDec("z3f")
    val CBC_ENCRYPTION_ALGORITHM = ClassDec("q47")
}
package com.jaqxues.sniptools.packimpl.hookdec

import com.jaqxues.akrolyb.genhook.decs.MemberDec.ConstructorDec
import com.jaqxues.akrolyb.genhook.decs.MemberDec.MethodDec
import com.jaqxues.sniptools.packimpl.features.*
import com.jaqxues.sniptools.packimpl.hookdec.ClassDeclarations.APP_START_EXPERIMENT_MANAGER
import com.jaqxues.sniptools.packimpl.hookdec.ClassDeclarations.CBC_ENCRYPTION_ALGORITHM
import com.jaqxues.sniptools.packimpl.hookdec.ClassDeclarations.SCREENSHOT_DETECTOR
import com.jaqxues.sniptools.packimpl.hookdec.ClassDeclarations.SNAP_MODEL
import com.jaqxues.sniptools.packimpl.hookdec.ClassDeclarations.STORY_VIEWE

# Interoptability with Akrolyb

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. Just paste the output (tuple of MethodDecs) in the corresponding section in this Notebook.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```